[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션 ID 설정하여, 대화 진행합니다. (muti-turn conversation)
LLM 모델이 과거 대화를 알지 (기억) 못하는 상황을 만드세요.

In [1]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)


In [4]:

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

from langchain_core.messages import trim_messages

## 6. trim 설정
trimmer = trim_messages(
    max_tokens = 565,      ## 최대 토큰 수
    strategy = 'last',     ## 최근 메시지부터 시작해서 자르기
    token_counter = llm,   ## 모델 기준으로 토큰 수 계산
    include_system= True,  ## 시스템 메시지 포함
    allow_partial= False,  ## 메시지 자르기 제한
    start_on= 'human',     ## human 메시지부터 자르기 시작
)

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

## 6. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history= itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
)

## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행

while True:
    query = input('육아휴직과 관련된 질문하세요 >> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )

    print('\n' + '*' * 50 + '\n')

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-I7YZBZAi6cSYieKqbDYr7iZd on tokens per min (TPM): Limit 30000, Used 29799, Requested 19033. Please try again in 37.664s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [22]:
store

{'abc': InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직이란?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직은 임신 중인 여성 근로자가 모성을 보호하거나, 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 근로자가 신청하는 휴직을 의미합니다(제19조(육아휴직), 제2절). 이 휴직은 근로자가 자녀의 양육을 위해 일정 기간 동안 근무를 중단하고 자녀양육에 전념할 수 있도록 마련된 제도입니다.\n\n육아휴직의 기간은 최대 1년 이며, 근로자가 신청할 경우 허용되어야 합니다(제19조(육아휴직) 제2항). 또한, 이미 사용한 육아휴직은 3회에 나누어 사용할 수 있고, 육아휴직 후에는 이전과 같은 업무 또는 같은 수준의 임금을 지급하는 직무에 복귀해야 하는 등 일정한 규정이 있습니다(제19조의4, 제22조의4).\n\n사업주는 육아휴직을 이유로 근로자를 해고하거나 불리한 처우를 할 수 없고, 육아휴직 기간 동안에는 근로자를 해고할 수 없으며, 복귀 시에는 같은 업무 또는 같은 수준의 임금이 지급되는 직무에 복귀시켜야 합니다(제19조(육아휴직) 제3항, 제22조의4).', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f'}, id='run-3b3d6c1f-5667-47a1-8dcf-59b6f14a3226-0')])}

In [23]:
for message in get_history('abc').messages:
    print(f'{[message.type.upper()]}: {message.content}\n')

['HUMAN']: 육아휴직이란?

['AI']: 육아휴직은 임신 중인 여성 근로자가 모성을 보호하거나, 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 근로자가 신청하는 휴직을 의미합니다(제19조(육아휴직), 제2절). 이 휴직은 근로자가 자녀의 양육을 위해 일정 기간 동안 근무를 중단하고 자녀양육에 전념할 수 있도록 마련된 제도입니다.

육아휴직의 기간은 최대 1년 이며, 근로자가 신청할 경우 허용되어야 합니다(제19조(육아휴직) 제2항). 또한, 이미 사용한 육아휴직은 3회에 나누어 사용할 수 있고, 육아휴직 후에는 이전과 같은 업무 또는 같은 수준의 임금을 지급하는 직무에 복귀해야 하는 등 일정한 규정이 있습니다(제19조의4, 제22조의4).

사업주는 육아휴직을 이유로 근로자를 해고하거나 불리한 처우를 할 수 없고, 육아휴직 기간 동안에는 근로자를 해고할 수 없으며, 복귀 시에는 같은 업무 또는 같은 수준의 임금이 지급되는 직무에 복귀시켜야 합니다(제19조(육아휴직) 제3항, 제22조의4).



1. 육아 휴직 기간은
2. 육아 휴직 조건은
3. 대상자는?
4. 수당은 얼마?

 첫번째 질문이 뭐야